# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import base64
import gradio as gr
import json
import os

from dotenv import load_dotenv
from io import BytesIO
from openai import OpenAI
from PIL import Image

In [2]:
MODEL = 'gpt-4o'

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
openai = OpenAI()

In [5]:
SYSTEM_MESSAGE = "You are a helpful AI assistant that is an expert on \
generating images in various art styles. You will help the user answer whatever questions \
they may have. If the user asks for any type of image, you will generate the image in the style they prefer.\
If it is not clear what style of image they would like, please ask for further details."

In [6]:
image_tool = {
    "name": "image_generator",
    "description": "This function should be called when the user is asking to create an image of any kind.",
    "parameters": {
        "type": "object",
        "properties": {
            "image_description": {
                "type": "string",
                "description": "The description of the image to be created",
            },
        },
        "required": ["image_description"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": image_tool}]

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    image_description = arguments.get('image_description')
    image = image_generator(image_description)
    tool_response = {
        "role": "tool",
        "content": json.dumps({"image_description": image_description}),
        "tool_call_id": tool_call.id
    }
    return tool_response, image

In [7]:
def image_generator(description):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=description,
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    image = Image.open(BytesIO(image_data))
    return image

In [8]:
def chat(history):
    messages = [{"role": "system", "content": SYSTEM_MESSAGE}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        tool_response, image = handle_tool_call(message)
        messages.append(message)
        messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    return history, image

In [9]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
